In [1]:
from clickhouse_driver import Client

import re
from tqdm import tqdm_notebook as tqdm

In [2]:
CLICKHOUSE_HOST = 'localhost'
COMPRESSION = 'lz4'
client = Client(host=CLICKHOUSE_HOST, compression=True)
execute = client.execute
FILE_FOULDER = './Data/'
CHUNK_SIZE = 1500000

In [3]:
filenames = !dir /b .\Data
filenames.remove('caligraph-instances_labels.nt') #only labels, not a graph
filenames

['caligraph-instances_dbpedia-mapping.nt',
 'caligraph-instances_relations.nt',
 'caligraph-instances_transitive-types.nt',
 'caligraph-instances_types.nt',
 'caligraph-metadata.nt',
 'caligraph-ontology.nt',
 'caligraph-ontology_dbpedia-mapping.nt',
 'caligraph-ontology_provenance.nt',
 'dbpedia_caligraph-instances.nt',
 'dbpedia_caligraph-relations.nt',
 'dbpedia_caligraph-types.nt']

In [4]:
%%time
p = re.compile(r"<\S*>")

def clear_trpl_string(triplet_str):  
    if triplet_str.find('#')!=-1:
        sep='#'
    else:
        sep='/'       
    return triplet_str.strip('><?|&!_.()%/').split(sep)[-1]


for file in tqdm(filenames):
    chunk = []
    for line in open(FILE_FOULDER + file, 'rt', encoding='utf-8'):
        triplet =  p.findall(line)
        
        if len(triplet)==3:
            subj, pred, obj = (clear_trpl_string(item) for item in triplet)
        
            if len(chunk)< CHUNK_SIZE:
                chunk.append((file, subj, pred, obj))
            else:
                execute('''INSERT INTO 
                            kg.caligraph 
                               (source, subject, 
                                   predicate, object) VALUES''', chunk )
                chunk = []
        else:
            continue
    
    #last run when exit for
    if len(chunk)>0:
        chunk.append((file, subj, pred, obj))
        execute('''INSERT INTO 
                kg.caligraph 
                   (source, subject, 
                       predicate, object) VALUES''', chunk )


Wall time: 1h 27min 15s
